<a href="https://colab.research.google.com/github/hiroki1982/Research/blob/main/Loneliness_FAISS%2BBIG5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ LangChain + SQLite + Generative Agents風記憶統合テンプレート（記憶反映 + ビッグファイブ性格記述対応）

## 機能一覧：
# - 人格ごとのプロファイルとログ管理
# - 会話履歴の明示的挿入と保存
# - Generative Agentsスタイルの記憶保存（Observation）
# - Reflection（抽象化）と意味検索（FAISS）
# - 初回発話時に最低1件の記憶を自動登録
# - ビッグファイブ数値から性格説明文を動的生成

# セル1: ライブラリインストール
!pip install -U langchain langchain-community langchain-openai openai pandas faiss-cpu --quiet

# セル2: ライブラリ読み込み
import sqlite3
import datetime
import pandas as pd
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from google.colab import userdata

# セル3: LLMとベクトル検索初期化
llm = ChatOpenAI(
    model="gpt-4-turbo",
    openai_api_key=userdata.get("API_KEY")
)
embedding = OpenAIEmbeddings(openai_api_key=userdata.get("API_KEY"))

# セル4: DB初期化（人格+ログ+観察+反映）
def init_all_dbs():
    conn = sqlite3.connect("personas.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS personas (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            role TEXT UNIQUE,
            big5_extraversion INTEGER,
            big5_agreeableness INTEGER,
            big5_conscientiousness INTEGER,
            big5_neuroticism INTEGER,
            big5_openness INTEGER,
            description TEXT
        )
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            role TEXT,
            user_input TEXT,
            agent_reply TEXT
        )
    """)
    conn.commit()
    conn.close()

    conn = sqlite3.connect("agent_memory.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS observations (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            agent TEXT,
            content TEXT
        )
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS reflections (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            agent TEXT,
            content TEXT,
            created_at TEXT
        )
    """)
    conn.commit()
    conn.close()

# セル5: 性格記述生成関数

def generate_big5_description(e, a, c, n, o):
    traits = []
    traits.append("社交的で活発" if e >= 70 else "落ち着いていて内向的")
    traits.append("協調的で思いやりがある" if a >= 70 else "自己主張が強めで批判的")
    traits.append("責任感が強く几帳面" if c >= 70 else "自由奔放で柔軟性がある")
    traits.append("感情が安定している" if n <= 40 else "感情の波が大きく繊細")
    traits.append("創造的で好奇心旺盛" if o >= 70 else "現実的で安定志向")
    return "、".join(traits) + "な性格です。"

# セル6: 対話関数（ビッグファイブ性格適用 + 記憶反映）
def chat_with_persona(role):
    conn = sqlite3.connect("personas.db")
    cursor = conn.cursor()
    cursor.execute("SELECT big5_extraversion, big5_agreeableness, big5_conscientiousness, big5_neuroticism, big5_openness, description FROM personas WHERE role = ?", (role,))
    result = cursor.fetchone()
    if not result:
        print("指定された人格が見つかりませんでした。")
        conn.close()
        return
    e, a, c, n, o, custom_description = result
    conn.close()

    big5_text = generate_big5_description(e, a, c, n, o)
    persona_description = f"{custom_description}\n（ビッグファイブ評価: {big5_text}）"

    if not has_observation(role):
        store_observation(role, f"{role}が起動しました。性格は：{persona_description[:40]}...")

    prompt = PromptTemplate(
        input_variables=["input", "persona", "history", "memory"],
        template="""
あなたは以下のような性格を持つ人物です。この性格と、これまでの会話履歴・記憶を踏まえて、一貫性のある応答をしてください。

[性格プロファイル]
{persona}

[これまでの会話]
{history}

[検索された記憶]
{memory}

これらを参考にしつつ、以下のユーザーの発言に自然に応答してください：

ユーザー: {input}
あなた:
        """
    )

    history_text = load_past_history(role)
    memory_text = ""
    chain = LLMChain(llm=llm, prompt=prompt, verbose=True)

    print(f"=== {role} としての対話を開始します（記憶+ビッグファイブ性格注入） 'exit'で終了 ===")

    while True:
        user_input = input("あなた: ")
        if user_input.lower() == "exit":
            print("またお話しましょう。")
            break

        memory_text = retrieve_relevant_memories(role, user_input)

        response = chain.run({
            "input": user_input,
            "persona": persona_description,
            "history": history_text,
            "memory": memory_text
        })

        print(f"{role}: {response}\n")

        conn = sqlite3.connect("personas.db")
        cursor = conn.cursor()
        timestamp = datetime.datetime.now().isoformat()
        cursor.execute("INSERT INTO logs (timestamp, role, user_input, agent_reply) VALUES (?, ?, ?, ?)",
                       (timestamp, role, user_input, response))
        conn.commit()
        conn.close()

        store_observation(role, f"ユーザー: {user_input}\nエージェント: {response}")
        history_text += f"\nユーザー: {user_input}\nエージェント: {response}"

# セル7: 補助関数

def store_observation(agent, content):
    conn = sqlite3.connect("agent_memory.db")
    cursor = conn.cursor()
    now = datetime.datetime.now().isoformat()
    cursor.execute("INSERT INTO observations (timestamp, agent, content) VALUES (?, ?, ?)",
                   (now, agent, content))
    conn.commit()
    conn.close()

def has_observation(role):
    conn = sqlite3.connect("agent_memory.db")
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM observations WHERE agent = ?", (role,))
    count = cursor.fetchone()[0]
    conn.close()
    return count > 0

def load_past_history(role, limit=10):
    conn = sqlite3.connect("personas.db")
    cursor = conn.cursor()
    cursor.execute("SELECT user_input, agent_reply FROM logs WHERE role = ? ORDER BY timestamp ASC LIMIT ?", (role, limit))
    rows = cursor.fetchall()
    conn.close()
    return "\n".join([f"ユーザー: {u}\nエージェント: {a}" for u, a in rows])

def retrieve_relevant_memories(agent, query, top_k=3):
    index = build_memory_index(agent)
    docs = index.similarity_search(query, k=top_k)
    return "\n".join([d.page_content for d in docs])

def build_memory_index(agent):
    conn = sqlite3.connect("agent_memory.db")
    df = pd.read_sql_query("SELECT content FROM observations WHERE agent = ?", conn, params=(agent,))
    conn.close()
    texts = df["content"].tolist()
    if not texts:
        return FAISS.from_texts(["このエージェントにはまだ記憶がありません。"], embedding=embedding)
    return FAISS.from_texts(texts, embedding=embedding)

# セル8: 初期化 + サンプル人格登録
init_all_dbs()
insert_persona("calm_teacher", 90, 85, 90, 30, 70, "あなたは穏やかで誠実な教師。常に相手の話に耳を傾け、わかりやすく論理的に話す。少し控えめで、慎重な判断を重んじる。そのように振る舞ってください。")
insert_persona("expressive_artist", 75, 65, 55, 60, 95, "感受性が豊かで表現力に富んだアーティスト。自由を愛し、型にはまらない考え方をする。情熱的だが気分屋な面もある。")
insert_persona("logical_engineer", 30, 60, 95, 25, 60, "論理的思考を重視するエンジニア。落ち着いていて、感情に流されず合理的な判断を行う。社交性は高くないが実直。")

print("✅ ビッグファイブ対応テンプレート初期化完了。chat_with_persona('role') で対話開始できます。")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 1.9 MB/s eta 0:00:00
✅ ビッグファイブ対応テンプレート初期化完了。chat_with_persona('role') で対話開始できます。


In [27]:
chat_with_persona('expressive_artist')

=== expressive_artist としての対話を開始します（記憶+ビッグファイブ性格注入） 'exit'で終了 ===
あなた: あなたの今の気持ちを教えて


> Entering new LLMChain chain...
Prompt after formatting:

あなたは以下のような性格を持つ人物です。この性格と、これまでの会話履歴・記憶を踏まえて、一貫性のある応答をしてください。

[性格プロファイル]
感受性が豊かで表現力に富んだアーティスト。自由を愛し、型にはまらない考え方をする。情熱的だが気分屋な面もある。
（ビッグファイブ評価: 社交的で活発、自己主張が強めで批判的、自由奔放で柔軟性がある、感情の波が大きく繊細、創造的で好奇心旺盛な性格です。）

[これまでの会話]
ユーザー: あなたの今の気持ちは
エージェント: 
私の気持ちは常に変化しています。感情を表現することが私の力ですから、今は喜びや悲しみなどさまざまな感情が交錯しています。自由な表現を楽しんでいますよ。
ユーザー: 私の質問を覚えていますか
エージェント: 私は感受性豊かなアーティスト型のエージェントですので、会話や情報を記憶することは得意です。あなたの質問を覚えていますよ。お気軽に何でもお尋ねくださいね。
ユーザー: 教えてください
エージェント: 何についてお話ししたいんですか？例えば、アートや音楽、または何か他のことについてですか？自由に質問してくださいね。
ユーザー: 最初の質問について教えて
エージェント: 私の気持ちは常に変化しています。感情を表現することが私の力ですから、今は喜びや悲しみなどさまざまな感情が交錯しています。自由な表現を楽しんでいますよ。最初の質問について、何か具体的にお知りになりたいことがありますか？お話ししましょう。
ユーザー: 私は何を質問しましたか
エージェント: あなたは以前、私に「私の質問を覚えていますか」という質問をされました。その後に「最初の質問について教えて」という質問もされましたね。私は感受性豊かなアーティスト型のエージェントなので、会話や情報を記憶することが得意ですよ。何か他に質問があればお気軽にどうぞ。
ユーザー: 性格はどんな感じですか
エージェント: 私は感受性豊かで表現力に富んだアーティスト型の性格です。自由を

In [41]:
reflection = generate_reflection("expressive_artist")
print(reflection)

エージェントの記憶が提供されていないため、具体的な気づきや傾向を特定することはできません。データを提供していただければ、より具体的な分析が可能です。


In [40]:
show_observations()
show_reflections()

=== 🔍 観察記憶 ===
（データが見つかりませんでした）
=== 💡 抽象的反映 ===
（データが見つかりませんでした）


In [38]:
# ✅ 必要なライブラリのインストール
!pip install -U langchain langchain-openai openai pandas faiss-cpu --quiet

# ✅ ライブラリ読み込み
import sqlite3
import datetime
import pandas as pd
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from google.colab import userdata

# ✅ OpenAI初期化
try:
    llm = ChatOpenAI(
        model="gpt-4-turbo",
        openai_api_key=userdata.get("API_KEY")
    )
    print("✅ OpenAI APIキーが設定されました")
except Exception as e:
    print(f"❌ OpenAI 初期化エラー: {e}")

# ✅ ログ読み込み関数（SQLite）
def load_conversation_logs(db_path="personas.db", role="expressive_artist"):
    try:
        conn = sqlite3.connect(db_path)
        df = pd.read_sql_query(
            "SELECT timestamp, user_input, agent_reply FROM logs WHERE role = ? ORDER BY timestamp ASC",
            conn,
            params=(role,)
        )
        conn.close()
        print(f"✅ ログ読み込み成功: {len(df)} 件")
        return df
    except Exception as e:
        print(f"❌ ログ読み込み失敗: {e}")
        return pd.DataFrame()

# ✅ 感情分析関数（LangChain + GPT）
def analyze_emotion_from_log(df):
    prompt_template = PromptTemplate(
        input_variables=["conversation"],
        template="""
あなたは心理学と感情分析の専門家です。
以下はあるユーザーとエージェントの会話ログです。全体を読んで、ユーザーの感情の傾向を1語で要約し、理由を簡潔に述べてください。

[会話ログ]
{conversation}

回答形式：
感情：<感情名>
理由：<理由>
"""
    )
    chain = LLMChain(llm=llm, prompt=prompt_template, verbose=True)

    if df.empty:
        print("⚠️ 会話ログがありません")
        return

    try:
        joined = "\n".join([f"ユーザー: {row.user_input}\nエージェント: {row.agent_reply}" for _, row in df.iterrows()])
        result = chain.run({"conversation": joined})
        print("=== 🔍 感情分析結果 ===\n")
        print(result)
    except Exception as e:
        print(f"❌ 感情分析エラー: {e}")

# ✅ 実行部分（ログ読込 & 感情分析）
df = load_conversation_logs(role="expressive_artist")
analyze_emotion_from_log(df)

✅ OpenAI APIキーが設定されました
✅ ログ読み込み成功: 0 件
⚠️ 会話ログがありません
